In [1]:
def printCapacities(capacities):
    maxLen = len(str(max([max(row) for row in capacities])))
    width = len(capacities)*maxLen + len(capacities) + 1
    for row in capacities:
        print("-"*width)
        print("|"+"|".join([str(capacities).center(maxLen) for capacities in row])+"|")
    print("-"*width)

In [4]:
#I will first augment the graph to add a single source and single drain (entrance and exit). The single
#source will have paths to the original sources with weights of each source's total outward flow.
#The new drain will have paths from the original drains in proportion to their total inbound flow.
#In this way, we can use a single source and single drain algorithm unmodified.

#Use bfs to find shortest augmenting path
def bfs(source, sink, capacities):
    queue = [(source, [source])]
    visited = [True] + [False] * len(capacities)
    while queue:
        current, path = queue.pop(0)
        if current == sink:
            return path
        for successor, capacity in enumerate(capacities[current]):
            if capacity > 0 and not visited[successor]:
                visited[successor] = True
                queue.append((successor, path + [successor]))
    return None

def solution(entrance, exit, capacities):
    #Compute row of new source
    sRow = []
    for i, row in enumerate(capacities):
        if i in entrance:
            sRow.append(sum(capacities[i]))
        else:
            sRow.append(0)
    #Compute col of new drain
    dCol = [0]
    for i in range(len(capacities)):
        if i in exit:
            dCol.append(sum([row[i] for row in capacities]))
        else:
            dCol.append(0)
    #Add new rows and cols to capacities
    capacities = [sRow] + capacities
    capacities = [[0] + row + [dCol[i]] for i, row in enumerate(capacities)]
    capacities = capacities + [[0]*len(capacities[0])]
    printCapacities(capacities)
    #Compute max flow from new source to new drain
    flow = 0
    source = 0
    sink = len(capacities)-1
    path = bfs(source, sink, capacities)
    while path:
        minCapacity = None
        for start, end in zip(path, path[1:]):
            currCapacity = capacities[start][end]
            minCapacity = currCapacity if not minCapacity or currCapacity < minCapacity else minCapacity
        for start, end in zip(path, path[1:]):
            print(f"modifying {start} to {end}")
            capacities[start][end] -= minCapacity
            capacities[end][start] += minCapacity
        flow += minCapacity
        path = bfs(source, sink, capacities)
        printCapacities(capacities)
        print(path)
        print(minCapacity)
    return flow

In [7]:
#TEST CASES
entrances = [[0], [0, 1], [0]]
exits = [[3], [4, 5], [3]]
paths = [[[0, 7, 0, 0], [0, 0, 6, 0], [0, 0, 0, 8], [9, 0, 0, 0]], [[0, 0, 4, 6, 0, 0], [0, 0, 5, 2, 0, 0], [0, 0, 0, 0, 4, 4], [0, 0, 0, 0, 6, 6], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]], [[0,2,0,0], [0,0,3,0], [0,0,0,1], [0,0,0,0]]]
expected = [6, 16, 1]
for entrance,exit,path,expect in zip(entrances,exits,paths,expected):
    print("TESTING PATH")
    printCapacities(path)
    print(f"ENTRANCES: {entrance}")
    print(f"EXITS: {exit}")
    actual = solution(entrance, exit, path)
    print(f"solution=>{actual}")
    if actual == expect:
        print("\tTEST PASSED")
    else:
        print(f"\tTEST FAILED, EXPECTED {expect}")

TESTING PATH
---------
|0|7|0|0|
---------
|0|0|6|0|
---------
|0|0|0|8|
---------
|9|0|0|0|
---------
ENTRANCES: [0]
EXITS: [3]
-------------
|0|7|0|0|0|0|
-------------
|0|0|7|0|0|0|
-------------
|0|0|0|6|0|0|
-------------
|0|0|0|0|8|0|
-------------
|0|9|0|0|0|8|
-------------
|0|0|0|0|0|0|
-------------
modifying 0 to 1
modifying 1 to 2
modifying 2 to 3
modifying 3 to 4
modifying 4 to 5
-------------
|0|1|0|0|0|0|
-------------
|6|0|1|0|0|0|
-------------
|0|6|0|0|0|0|
-------------
|0|0|6|0|2|0|
-------------
|0|9|0|6|0|2|
-------------
|0|0|0|0|6|0|
-------------
None
6
solution=>6
	TEST PASSED
TESTING PATH
-------------
|0|0|4|6|0|0|
-------------
|0|0|5|2|0|0|
-------------
|0|0|0|0|4|4|
-------------
|0|0|0|0|6|6|
-------------
|0|0|0|0|0|0|
-------------
|0|0|0|0|0|0|
-------------
ENTRANCES: [0, 1]
EXITS: [4, 5]
-------------------------
|0 |10|7 |0 |0 |0 |0 |0 |
-------------------------
|0 |0 |0 |4 |6 |0 |0 |0 |
-------------------------
|0 |0 |0 |5 |2 |0 |0 |0 |
-------